In [32]:
import pandas as pd
import numpy as np
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [33]:
train_df = pd.read_csv('C:/Users/User/Desktop/work_projects/brain_signals/7/far_near_imagery_one_person_train.csv', sep=';')
test_df = pd.read_csv('C:/Users/User/Desktop/work_projects/brain_signals/7/far_near_imagery_one_person_test.csv', sep=';')

In [34]:
train_df.head()

,time,condition,epoch,FC5,F3,FCz,F4,FC6,C5,C3,...,CP4,CP1,CP2,FC1,FC2,FC3,FC4,C1,C2,EMG
0,-1500,flash_far,0,3.887543,3.973868,1.966108,3.733231,0.757906,2.764968,0.708987,...,-2.215013,-2.218470,-1.179149,1.322782,1.468610,1.729086,1.588174,-1.297363,-1.002271,257.193835
1,-1498,flash_far,0,4.235073,4.000529,2.216059,4.125958,0.914313,2.713175,0.473620,...,-2.411684,-2.374950,-1.178508,1.288190,1.469939,1.570809,1.645112,-1.515293,-1.198473,219.642947
2,-1496,flash_far,0,4.557960,3.974904,2.481806,4.565713,1.185329,2.507814,0.145941,...,-2.554149,-2.558032,-1.138880,1.218727,1.482167,1.362568,1.744303,-1.763673,-1.386169,168.084960
3,-1494,flash_far,0,4.833803,3.888315,2.745796,5.008233,1.553867,2.150732,-0.249696,...,-2.642897,-2.739798,-1.061797,1.120624,1.498655,1.122784,1.863515,-2.011065,-1.556820,99.241655
4,-1492,flash_far,0,5.044827,3.738267,2.989370,5.399617,1.983206,1.666584,-0.674632,...,-2.691362,-2.885130,-0.953913,1.005940,1.508396,0.877459,1.968125,-2.218723,-1.706211,15.795218


In [35]:
# Assuming df is your DataFrame
unique_conditions = train_df['condition'].unique()
print(unique_conditions)


['flash_far' 'Flash_near']


In [36]:
import pandas as pd

# Step 1: Read the CSV file (assuming train_df is already loaded)

# Step 2: Get the number of channels (i.e., the number of columns related to channels)
# The columns start from index 3 onward for the channel columns
channel_columns = train_df.columns[3:]

# Step 3: Create a list to hold the train_dfs_raw for each channel
train_dfs_raw = []

# Iterate through each channel column
for channel in channel_columns:
    # Group by 'epoch' and extract the time and channel values for each epoch
    channel_data = train_df[['time', 'epoch', 'condition', channel]]
    
    # Pivot the data: each epoch will become a row, and each time step will become a column
    channel_pivot = channel_data.pivot_table(index='epoch', columns='time', values=channel, aggfunc='first')
    
    # Optionally, include the 'condition' (action) column in the DataFrame
    # Use the first value of 'condition' for each epoch (this should be the same within an epoch)
    condition_data = train_df.groupby('epoch')['condition'].first()
    channel_pivot['class'] = condition_data  # Rename 'action' to 'class'
    
    # Replace 'active' with 1 and 'imagery' with 0 in the 'class' column
    channel_pivot['class'] = channel_pivot['class'].replace({'flash_far': 1, 'Flash_near': 0})
    
    # Drop the first column (index 0) and reset the index to avoid keeping 'epoch' as index
    channel_pivot = channel_pivot.reset_index(drop=True)
    
    # Drop the row with the time ticks (which is typically the first row in the DataFrame)
    channel_pivot.columns.name = None  # Remove the extra column header name (time ticks)
    
    # Append this channel's dataframe to the list
    train_dfs_raw.append(channel_pivot)

# Step 4: Verify the result for the first channel's dataframe
train_dfs_raw[0].head()  # This will print the first few rows of the first channel's dataframe



C:\Users\User\AppData\Local\Temp\ipykernel_29816\2655227071.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  channel_pivot['class'] = channel_pivot['class'].replace({'flash_far': 1, 'Flash_near': 0})
C:\Users\User\AppData\Local\Temp\ipykernel_29816\2655227071.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  channel_pivot['class'] = channel_pivot['class'].replace({'flash_far': 1, 'Flash_near': 0})
C:\Users\User\AppData\Local\Temp\ipykernel_29816\2655227071.py:26: FutureWarning: Downcasting behavior in `replace` is deprecate

,-1500,-1498,-1496,-1494,-1492,-1490,-1488,-1486,-1484,-1482,...,1984,1986,1988,1990,1992,1994,1996,1998,2000,class
0,3.887543,4.235073,4.557960,4.833803,5.044827,5.179409,5.232629,5.205817,5.105183,4.939806,...,-0.144343,-0.568515,-0.865764,-1.031828,-1.076297,-1.019924,-0.890873,-0.720471,-0.539071,1
1,-1.639928,-1.554213,-1.366475,-1.077605,-0.702663,-0.270201,0.180568,0.604531,0.956790,1.199315,...,2.101390,2.398104,2.588234,2.658747,2.610315,2.456568,2.221716,1.936938,1.636095,1
2,3.372979,3.695626,3.867618,3.826896,3.528685,2.953315,2.111304,1.044806,-0.175021,-1.454614,...,-5.560480,-5.581408,-5.446869,-5.168105,-4.770666,-4.290965,-3.771632,-3.256324,-2.784702,1
3,-0.298652,-0.138541,0.039754,0.224836,0.403800,0.563400,0.691261,0.776969,0.812949,0.795038,...,-0.948014,-1.164240,-1.344137,-1.499472,-1.640479,-1.774754,-1.906589,-2.036812,-2.163145,1
4,-1.099402,-0.884351,-0.648879,-0.405201,-0.167071,0.051569,0.238194,0.383404,0.482564,0.537181,...,-1.327552,-1.295377,-1.251674,-1.194308,-1.122013,-1.034865,-0.934580,-0.824581,-0.709789,1


тест

In [37]:

# Assuming test_df is your test DataFrame without the 'condition' column# Step 2: Get the number of channels (i.e., the number of columns related to channels)
# The columns start from index 3 onward for the channel columns
channel_columns = test_df.columns[2:]

# Step 3: Create a list to hold the test_dfs_raw for each channel
test_dfs_raw = []

# Iterate through each channel column
for channel in channel_columns:
    # Group by 'epoch' and extract the time and channel values for each epoch
    channel_data = test_df[['time', 'epoch', channel]]
    
    # Pivot the data: each epoch will become a row, and each time step will become a column
    channel_pivot = channel_data.pivot_table(index='epoch', columns='time', values=channel, aggfunc='first')
    
    # If 'condition' column exists, add 'class' column (you can add your specific logic here)
    if 'condition' in test_df.columns:
        # Use the first value of 'condition' for each epoch (this should be the same within an epoch)
        condition_data = test_df.groupby('epoch')['condition'].first()
        channel_pivot['class'] = condition_data  # Rename 'action' to 'class'
        
        # Replace 'active' with 1 and 'imagery' with 0 in the 'class' column
        channel_pivot['class'] = channel_pivot['class'].replace({'flash_far': 1, 'Flash_near': 0})
    
    # Drop the first column (index 0) and reset the index to avoid keeping 'epoch' as index
    channel_pivot = channel_pivot.reset_index(drop=True)
    
    # Drop the row with the time ticks (which is typically the first row in the DataFrame)
    channel_pivot.columns.name = None  # Remove the extra column header name (time ticks)
    
    # Append this channel's dataframe to the list
    test_dfs_raw.append(channel_pivot)

# Step 4: Verify the result for the first channel's dataframe
test_dfs_raw[0].head()  # This will print the first few rows of the first channel's dataframe



,-1500,-1498,-1496,-1494,-1492,-1490,-1488,-1486,-1484,-1482,...,1982,1984,1986,1988,1990,1992,1994,1996,1998,2000
0,-2.182410,-1.978824,-1.767805,-1.558360,-1.359017,-1.177243,-1.018973,-0.888227,-0.786795,-0.713992,...,1.288345,0.889545,0.550083,0.297274,0.147068,0.103205,0.157801,0.293235,0.485035,0.705337
1,2.079987,2.626048,3.240505,3.877687,4.484147,5.004580,5.388476,5.596635,5.606632,5.416416,...,-4.484237,-3.436134,-2.388724,-1.392487,-0.492484,0.276202,0.891614,1.346340,1.647085,1.812603
2,0.164457,0.159236,0.221174,0.364828,0.597380,0.916874,1.311556,1.760473,2.235339,2.703457,...,-1.838490,-0.676968,0.450076,1.462528,2.291920,2.889603,3.232210,3.323707,3.193771,2.892847
3,-0.897161,-1.085759,-1.290258,-1.482748,-1.630729,-1.700899,-1.663558,-1.497061,-1.191705,-0.752459,...,-7.062761,-6.850382,-6.483484,-5.967164,-5.322418,-4.584086,-3.797005,-3.010912,-2.274765,-1.631277
4,-3.254901,-4.452717,-5.681391,-6.838672,-7.823854,-8.549548,-8.952048,-8.998858,-8.692353,-8.069073,...,-3.421909,-3.422411,-3.343742,-3.192209,-2.976939,-2.709864,-2.405495,-2.080391,-1.752272,-1.438823


In [38]:
train_dfs_raw[0].head()

,-1500,-1498,-1496,-1494,-1492,-1490,-1488,-1486,-1484,-1482,...,1984,1986,1988,1990,1992,1994,1996,1998,2000,class
0,3.887543,4.235073,4.557960,4.833803,5.044827,5.179409,5.232629,5.205817,5.105183,4.939806,...,-0.144343,-0.568515,-0.865764,-1.031828,-1.076297,-1.019924,-0.890873,-0.720471,-0.539071,1
1,-1.639928,-1.554213,-1.366475,-1.077605,-0.702663,-0.270201,0.180568,0.604531,0.956790,1.199315,...,2.101390,2.398104,2.588234,2.658747,2.610315,2.456568,2.221716,1.936938,1.636095,1
2,3.372979,3.695626,3.867618,3.826896,3.528685,2.953315,2.111304,1.044806,-0.175021,-1.454614,...,-5.560480,-5.581408,-5.446869,-5.168105,-4.770666,-4.290965,-3.771632,-3.256324,-2.784702,1
3,-0.298652,-0.138541,0.039754,0.224836,0.403800,0.563400,0.691261,0.776969,0.812949,0.795038,...,-0.948014,-1.164240,-1.344137,-1.499472,-1.640479,-1.774754,-1.906589,-2.036812,-2.163145,1
4,-1.099402,-0.884351,-0.648879,-0.405201,-0.167071,0.051569,0.238194,0.383404,0.482564,0.537181,...,-1.327552,-1.295377,-1.251674,-1.194308,-1.122013,-1.034865,-0.934580,-0.824581,-0.709789,1


In [39]:
# train_dfs_raw[0].shape[1]
test_dfs_raw[0].shape[1]

1751

In [40]:
train_dfs_raw_cut = []

for df in train_dfs_raw:
    df = df.iloc[:, 500:-500]
    train_dfs_raw_cut.append(df)

train_df_raw_cut_0 = train_dfs_raw_cut[0]

тест

In [41]:
test_dfs_raw_cut = []

for df in test_dfs_raw:
    df = df.iloc[:, 500:-499]
    test_dfs_raw_cut.append(df)

test_df_raw_cut_0 = test_dfs_raw_cut[0]

In [42]:
# train_dfs_raw_cut[0].head()
train_dfs_raw_cut[0].shape[1]

752

In [43]:
window_size = 25

def smoothing_row(df, window_size):
    smoothed_df = df.apply(lambda row: row.rolling(window=window_size, min_periods=1).mean(), axis=1)
    return smoothed_df

train_dfs_raw_cut_smooth = [smoothing_row(df, window_size) for df in train_dfs_raw_cut]



тест

In [44]:
window_size = 25

def smoothing_row(df, window_size):
    smoothed_df = df.apply(lambda row: row.rolling(window=window_size, min_periods=1).mean(), axis=1)
    return smoothed_df

test_dfs_raw_cut_smooth = [smoothing_row(df, window_size) for df in test_dfs_raw_cut]


# Нейросеть 

In [45]:
# Convert each dataframe to a NumPy array and stack them
data_arrays = [df.values for df in train_dfs_raw_cut_smooth]  # List of arrays with shape [500, 150]
data = np.stack(data_arrays, axis=1)  # Shape: [500, 6, 150]

# Convert labels to NumPy array (ensure it's binary: 0 or 1)
labels = train_dfs_raw[0].iloc[:, -1].values  # Shape: [500]

In [46]:
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [47]:
class SignalDataset(Dataset):
    def __init__(self, data, labels):
        """
        Args:
            data (numpy.ndarray): Array of shape [num_samples, channels, time_steps]
            labels (numpy.ndarray): Array of shape [num_samples]
        """
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [48]:
from torch.utils.data import DataLoader, Subset
import numpy as np
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42, stratify=labels
)

# Create Dataset instances
train_dataset = SignalDataset(X_train, y_train)
test_dataset = SignalDataset(X_test, y_test)

# Now, let's balance the test dataset
# Get indices for each class
class_0_indices = np.where(y_test == 0)[0]
class_1_indices = np.where(y_test == 1)[0]

# We will take the minimum length of both classes to ensure balance
min_class_size = min(len(class_0_indices), len(class_1_indices))

# Select a random subset of indices from each class to balance the test dataset
class_0_sampled = np.random.choice(class_0_indices, min_class_size, replace=False)
class_1_sampled = np.random.choice(class_1_indices, min_class_size, replace=False)

# Combine the indices
balanced_indices = np.concatenate([class_0_sampled, class_1_sampled])

# Create a balanced test dataset
balanced_test_dataset = Subset(test_dataset, balanced_indices)

# Create DataLoader for the balanced test set
batch_size = 32
test_loader = DataLoader(balanced_test_dataset, batch_size=batch_size, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [49]:
class CNN_LSTM_Model(nn.Module):
    def __init__(self, num_channels=29, time_steps=752, cnn_out_channels=128, 
                 cnn_kernel_size=3, lstm_hidden_size=128, lstm_num_layers=2, 
                 num_classes=2, dropout=0.5):
        super(CNN_LSTM_Model, self).__init__()
        
        # CNN component
        self.conv1 = nn.Conv1d(in_channels=num_channels, out_channels=cnn_out_channels, kernel_size=cnn_kernel_size, padding=1)
        self.bn1 = nn.BatchNorm1d(cnn_out_channels)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        
        # Calculate the new time_steps after Conv and Pooling
        self.conv_output_time_steps = time_steps // 2  # After MaxPool with kernel_size=2
        
        # LSTM component
        self.lstm = nn.LSTM(input_size=cnn_out_channels, 
                            hidden_size=lstm_hidden_size, 
                            num_layers=lstm_num_layers, 
                            batch_first=True, 
                            dropout=dropout)
        
        # Fully connected layer
        self.fc = nn.Linear(lstm_hidden_size, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        """
        Args:
            x: Tensor of shape [batch_size, channels, time_steps]
        Returns:
            out: Tensor of shape [batch_size, num_classes]
        """
        # CNN layers
        x = self.conv1(x)  # [batch, cnn_out_channels, time_steps]
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)  # [batch, cnn_out_channels, time_steps/2]
        
        # Prepare for LSTM: transpose to [batch, time_steps, features]
        x = x.permute(0, 2, 1)  # [batch, time_steps/2, cnn_out_channels]
        
        # LSTM layers
        lstm_out, (hn, cn) = self.lstm(x)  # lstm_out: [batch, time_steps/2, lstm_hidden_size]
        
        # Take the last time step's output
        out = lstm_out[:, -1, :]  # [batch, lstm_hidden_size]
        
        out = self.dropout(out)
        out = self.fc(out)  # [batch, num_classes]
        
        return out

In [50]:
import torch
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Assuming `labels` is already defined (e.g., the target labels for training)
# Convert labels to a numpy array if they are not already
labels = np.array(labels)

# Calculate class weights (use sklearn.utils.class_weight)
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Instantiate the model
model = CNN_LSTM_Model()
model.to(device)

# Use weighted loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Define optimizer (Adam optimizer with a learning rate of 1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

Using device: cpu


In [51]:
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import numpy as np

num_epochs = 20  # Adjust based on convergence
best_auc = 0.0
best_model_path = 'best_cnn_lstm_model_auc.pth'

# Outer loop with tqdm for epoch tracking
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    all_preds_prob_train = []  # Store predicted probabilities on training data
    all_targets_train = []  # Store true labels on training data
    
    # Use tqdm for batch-level progress
    with tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as batch_progress:
        for batch_idx, (signals, targets) in enumerate(batch_progress):
            signals = signals.to(device)  # [batch, 6, 150]
            targets = targets.to(device)  # [batch]
            
            # Forward pass
            outputs = model(signals)  # [batch, 2]
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Store predicted probabilities (softmax outputs) for AUC calculation
            prob = torch.softmax(outputs, dim=1)  # [batch, 2]
            all_preds_prob_train.extend(prob.cpu().detach().numpy())  # Store in CPU for AUC calculation
            all_targets_train.extend(targets.cpu().numpy())  # Store true labels in CPU
            
            # Update tqdm with current loss for each batch
            batch_progress.set_postfix(loss=running_loss / (batch_idx + 1))
    
    avg_loss = running_loss / len(train_loader)

    # Calculate AUC on the training data (to track training progress)
    auc_train = roc_auc_score(all_targets_train, np.array(all_preds_prob_train)[:, 1])  # AUC for the positive class (class 1)
    
    # Print AUC and loss for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Train AUC: {auc_train:.4f}')
    
    # Evaluate on the test set for AUC after every epoch
    model.eval()
    all_preds_prob_test = []  # Store predicted probabilities on test data
    all_targets_test = []  # Store true labels on test data

    with torch.no_grad():
        for signals, targets in test_loader:
            signals = signals.to(device)
            targets = targets.to(device)
            
            outputs = model(signals)
            prob = torch.softmax(outputs, dim=1)  # [batch, 2]
            
            all_preds_prob_test.extend(prob.cpu().numpy())
            all_targets_test.extend(targets.cpu().numpy())
    
    # Calculate AUC on the test data
    auc_test = roc_auc_score(all_targets_test, np.array(all_preds_prob_test)[:, 1])
    print(f'Epoch [{epoch+1}/{num_epochs}], Test AUC: {auc_test:.4f}')
    
    # Save the model with the best AUC on the test set
    if auc_test > best_auc:
        best_auc = auc_test
        torch.save(model.state_dict(), best_model_path)
        print(f'Best model saved with AUC: {best_auc:.4f}')

# After training, load the best model and evaluate it on the test data
model.load_state_dict(torch.load(best_model_path))
model.eval()

# Final evaluation on the test set (for the record)
all_preds_prob_test_final = []
all_targets_test_final = []

with torch.no_grad():
    for signals, targets in test_loader:
        signals = signals.to(device)
        targets = targets.to(device)
        
        outputs = model(signals)
        prob = torch.softmax(outputs, dim=1)  # [batch, 2]
        
        all_preds_prob_test_final.extend(prob.cpu().numpy())
        all_targets_test_final.extend(targets.cpu().numpy())

# Calculate final AUC on the test data
final_auc_test = roc_auc_score(all_targets_test_final, np.array(all_preds_prob_test_final)[:, 1])
print(f'Final Test AUC: {final_auc_test:.4f}')

Epoch 1/20: 100%|██████████| 9/9 [00:13<00:00,  1.53s/batch, loss=0.717]


Epoch [1/20], Loss: 0.7168, Train AUC: 0.4590
Epoch [1/20], Test AUC: 0.5791
Best model saved with AUC: 0.5791


Epoch 2/20: 100%|██████████| 9/9 [00:12<00:00,  1.43s/batch, loss=0.704]


Epoch [2/20], Loss: 0.7040, Train AUC: 0.4555
Epoch [2/20], Test AUC: 0.5898
Best model saved with AUC: 0.5898


Epoch 3/20: 100%|██████████| 9/9 [00:11<00:00,  1.25s/batch, loss=0.702]


...

Epoch [18/20], Loss: 0.3721, Train AUC: 0.9333
Epoch [18/20], Test AUC: 0.9824


Epoch 19/20: 100%|██████████| 9/9 [00:02<00:00,  3.85batch/s, loss=0.219]


Epoch [19/20], Loss: 0.2187, Train AUC: 0.9611
Epoch [19/20], Test AUC: 0.9922


Epoch 20/20: 100%|██████████| 9/9 [00:02<00:00,  3.90batch/s, loss=0.256]
C:\Users\User\AppData\Local\Temp\ipykernel_29816\2797811246.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experiment

Epoch [20/20], Loss: 0.2557, Train AUC: 0.9715
Epoch [20/20], Test AUC: 0.9971
Final Test AUC: 1.0000


In [52]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Load the best model
model.load_state_dict(torch.load('best_cnn_lstm_model_auc.pth'))
model.eval()

# Final evaluation on the test set
all_preds = []
all_targets = []

with torch.no_grad():
    for signals, targets in test_loader:
        signals = signals.to(device)
        targets = targets.to(device)
        
        outputs = model(signals)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

# Calculate final accuracy
final_accuracy = accuracy_score(all_targets, all_preds)

# Calculate confusion matrix
final_cm = confusion_matrix(all_targets, all_preds)

# Extract TP, TN, FP, FN
TN, FP, FN, TP = final_cm.ravel()

# Print the accuracy and confusion matrix in a readable way
print(f'Final Test Accuracy: {final_accuracy:.4f}')
print(f'Final Confusion Matrix (readable format):')
print(f'True Negative (TN): {TN}')
print(f'False Positive (FP): {FP}')
print(f'False Negative (FN): {FN}')
print(f'True Positive (TP): {TP}')

# Classification Report (precision, recall, F1-score)
print("\nClassification Report:\n", classification_report(all_targets, all_preds))

# Optionally, you can print the confusion matrix in the usual format as well
print(f'\nConfusion Matrix (standard format):\n{final_cm}')

# Create the Comparison Table: True Labels vs Predicted Labels
comparison_df = pd.DataFrame({
    'True Label': all_targets,
    'Predicted Label': all_preds
})

print("\nComparison of True and Predicted Labels (first 10 rows):")
print(comparison_df.head(10))  # Display the first 10 rows for inspection


C:\Users\User\AppData\Local\Temp\ipykernel_29816\1752453437.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_cnn_lstm_model_auc.pth

Final Test Accuracy: 1.0000
Final Confusion Matrix (readable format):
True Negative (TN): 32
False Positive (FP): 0
False Negative (FN): 0
True Positive (TP): 32

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        32
           1       1.00      1.00      1.00        32

    accuracy                           1.00        64
   macro avg       1.00      1.00      1.00        64
weighted avg       1.00      1.00      1.00        64


Confusion Matrix (standard format):
[[32  0]
 [ 0 32]]

Comparison of True and Predicted Labels (first 10 rows):
   True Label  Predicted Label
0           0                0
1           0                0
2           0                0
3           0                0
4           0                0
5           0                0
6           0                0
7           0                0
8           0                0
9           0                0


# Предсказание

In [53]:
# Convert each dataframe in test_dfs_raw_smooth to a NumPy array and stack them
test_data_arrays = [df.values for df in test_dfs_raw_cut_smooth]  # List of arrays with shape [N, 150]
test_data = np.stack(test_data_arrays, axis=1)  # Shape: [N, 6, 150]

# Create a Dataset instance for test-only (without labels)
class TestOnlyDataset(Dataset):
    def __init__(self, data):
        """
        Args:
            data (numpy.ndarray): Array of shape [num_samples, channels, time_steps]
        """
        self.data = torch.tensor(data, dtype=torch.float32)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Create a DataLoader for `test_dfs_raw_smooth` (features only)
test_only_dataset = TestOnlyDataset(test_data)
batch_size = 32
test_only_loader = DataLoader(test_only_dataset, batch_size=batch_size, shuffle=False)

# Use the trained model to make predictions on the test data (features only)
model.eval()  # Set model to evaluation mode

all_preds = []

# Iterate over the test-only loader and make predictions
with torch.no_grad():
    for signals in test_only_loader:
        signals = signals.to(device)  # Move to GPU if needed
        
        outputs = model(signals)  # Get model output
        _, preds = torch.max(outputs, 1)  # Get predicted class (0 or 1)
        
        all_preds.extend(preds.cpu().numpy())  # Store predictions on CPU

# all_preds now contains all the predicted class labels (0 or 1)
print(f'Predicted classes for the test set: {all_preds}')


Predicted classes for the test set: [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]


In [54]:
folder_path = 'C:/Users/User/Desktop/work_projects/brain_signals/7'

In [55]:
import os
# Ensure the folder exists, create it if necessary
os.makedirs(folder_path, exist_ok=True)


# Convert predictions to binary string
binary_string = ''.join([str(pred) for pred in all_preds])

# Define the full file path
file_path = os.path.join(folder_path, 'predictions.csv')

# Write the binary string to the CSV file without any headers or index
with open(file_path, 'w') as f:
    f.write(binary_string)

print(f"File saved to: {file_path}")

File saved to: C:/Users/User/Desktop/work_projects/brain_signals/7\predictions.csv


In [56]:
answer = pd.read_csv('C:/Users/User/Desktop/work_projects/brain_signals/7/predictions.csv')

In [57]:
answer

,100001000100101101111101001111000111
